In [1]:
# Importing PyTorch
import torch
torch.version.__version__

'2.1.0+cpu'

# Loading training & testing dataset

In [2]:
import pandas as pd  # Import the Pandas library for data manipulation.

# Define the input file name as a variable for easy modification.
input_file = 'output_results.csv'

# Read the CSV data from the specified input file into a Pandas DataFrame.
csv_data = pd.read_csv(input_file)

# Display the first few rows of the DataFrame to inspect the data.
csv_data.head()

,H,B,q,R_int,gamma,Su,sf_lower,sf_upper
0,8,4,10,0.6,18,25,1.041,1.087
1,8,4,10,0.8,18,25,1.044,1.087
2,6,20,20,0.8,18,25,1.045,1.080
3,6,20,20,0.6,18,25,1.047,1.080
4,6,20,20,1.0,18,25,1.048,1.080


# Defining MyDataset class

## 1. Class definition

In [3]:
from torch.utils.data import Dataset
import numpy as np
import math

class MyDataset(Dataset):
    """
    A custom PyTorch dataset for loading data from a CSV file and preparing it for training or testing.

    Args:
        file_name (str): The path to the CSV file containing the dataset.
        train_test_ratio (float, optional): The ratio of the dataset to be used for training (default is 0.9).
        test (bool, optional): If True, the dataset is prepared for testing; otherwise, it's prepared for training (default is False).

    Attributes:
        x_data (torch.Tensor): The input data.
        y_data (torch.Tensor): The target data.

    Methods:
        __len__(): Returns the number of samples in the dataset.
        __getitem__(idx): Returns the input and target data for a specific index.
    """

    def __init__(self, file_name, train_test_ratio=0.95, test=False):
        """
        Initializes the MyDataset with data from a CSV file.

        Args:
            file_name (str): The path to the CSV file containing the dataset.
            train_test_ratio (float, optional): The ratio of the dataset to be used for training (default is 0.9).
            test (bool, optional): If True, the dataset is prepared for testing; otherwise, it's prepared for training (default is False).
        """
        _df = pd.read_csv(file_name)

        if test:
            data_len = math.floor((1 - train_test_ratio) * len(_df.iloc[:, 0]))
        else:
            data_len = math.floor(train_test_ratio * len(_df.iloc[:, 0]))

        x = _df.iloc[:data_len, :-2].values
        y = _df.iloc[:data_len, -2:].values

        self.x_data = torch.tensor(x, dtype=torch.float32)
        self.y_data = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return np.shape(self.y_data)[0]

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

## 2. MyDataset class implementation

In [4]:
# Create a training dataset from the input CSV file.
train_dataset = MyDataset(input_file)

# Create a testing dataset from the input CSV file.
test_dataset = MyDataset(input_file, test=True)

**Hyperparameters:**

- Number of training epochs: `EPOCH = 10`
- Batch size for mini-batch gradient descent: `BATCH_SIZE = 10`
- Learning rate for the optimization algorithm (e.g., stochastic gradient descent): `LR = 0.001`


In [5]:
# Explanation:
# - PyTorch uses random numbers in various operations, such as weight initialization, data shuffling, and more.
# - By setting the random seed to a specific value (in this case, 1), we ensure that the same random numbers are generated every time we run the code.
# - This is important for reproducibility, which means getting the same results each time you run the code.
# - Reproducibility is crucial in machine learning and deep learning to validate and compare results.

# Note: Setting the random seed to 1 is just an example. You can choose any integer value for reproducibility.
torch.manual_seed(1)  # Reproducible

# Number of training epochs
EPOCH = 10

# Batch size for mini-batch gradient descent
BATCH_SIZE = 10

# Learning rate for the optimization algorithm (e.g., stochastic gradient descent)
LR = 0.001

**Data Loaders and Batch Information:**

- Import the necessary DataLoader module from PyTorch to handle dataset loading.

- Create a data loader for the training dataset with the following parameters:
  - Batch size: `BATCH_SIZE`
  - No shuffle.

- Create a data loader for the test dataset with the same parameters as the training data loader.

- To obtain a single batch of training data and labels, use the `next` function on an iterator created from `train_loader`.

- Display the shape of the feature batch:
  - `train_features.size()` returns the shape (dimensions) of the feature batch.

- Display the shape of the label batch:
  - `train_labels.size()` returns the shape (dimensions) of the label batch.


In [6]:
from torch.utils.data import DataLoader

# Create a data loader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Create a data loader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Obtain a single batch of training data and labels using the `train_loader`.
# This is done by using the `next` function on an iterator created from `train_loader`.
train_features, train_labels = next(iter(train_loader))

# Print the shape of the feature batch.
# `train_features.size()` returns the shape (dimensions) of the feature batch.
print(f"Feature batch shape: {train_features.size()}")

# Print the shape of the label batch.
# `train_labels.size()` returns the shape (dimensions) of the label batch.
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([10, 6])
Labels batch shape: torch.Size([10, 2])


### Multiple Linear Regression Neural Network

Defines a PyTorch neural network module for multiple linear regression. The neural network consists of three fully connected layers, and the docstring provides a detailed explanation of its structure and usage.

In [7]:
import torch as T
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=6, hidden_size=32, batch_first=True)
        self.fc = nn.Linear(32, 2)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

### Initialize the Multiple Linear Regression (MLR) Model

An instance of the Multiple Linear Regression (MLR) model is created and its parameters are printed. The MLR model is designed for regression tasks, and in this instance, it is configured with 6 input features and is expected to produce 2 output values.


In [8]:
# Create an instance of the MultipleLinearRegression model with 6 input features and 2 output units.
MLR_model = RNN()

# Print the parameters of the MLR model.
print("The parameters: ", list(MLR_model.parameters()))

The parameters:  [Parameter containing:
tensor([[-0.0343,  0.0830, -0.1664,  0.1060, -0.0364,  0.0899],
        [ 0.0246, -0.0216,  0.0490,  0.0087,  0.0646, -0.0689],
        [-0.0129, -0.0159,  0.0256, -0.0007,  0.1545,  0.0550],
        [-0.0658, -0.1068, -0.0296, -0.0763, -0.0566,  0.0085],
        [ 0.1054,  0.0961, -0.1728,  0.1096,  0.0494,  0.1677],
        [ 0.1167, -0.1611, -0.1681, -0.0853,  0.1552, -0.0294],
        [ 0.0757, -0.0822,  0.1735, -0.0748,  0.1326,  0.0021],
        [-0.0931,  0.0909, -0.0938,  0.0520, -0.0510, -0.0194],
        [-0.1700, -0.0843,  0.0959, -0.0430,  0.1761,  0.1417],
        [-0.0083, -0.1180,  0.1077,  0.0549, -0.1143,  0.1148],
        [ 0.1073,  0.1568, -0.0991, -0.0291, -0.0034,  0.0258],
        [-0.1342, -0.1254,  0.0962, -0.0415,  0.0864,  0.0101],
        [ 0.0580,  0.0389,  0.0643,  0.0876, -0.1637,  0.0890],
        [-0.1243, -0.1334,  0.0108, -0.0301,  0.1038, -0.1024],
        [-0.1572,  0.1287, -0.0262,  0.0994,  0.0568, -0.1326],


# Model Optimization and Loss Definition

In this section, the model optimizer is selected, and the loss criterion for training the Multiple Linear Regression model is defined.

## Model Optimizer
We use the AdamW optimizer to update the model parameters during training. The learning rate (LR) determines the step size for parameter updates.

In [9]:
optimizer = torch.optim.AdamW(MLR_model.parameters(), lr=LR)

In [10]:
# Define the Mean Squared Error (MSE) loss criterion
criterion = torch.nn.MSELoss()

# Model Prediction Check (Before Training)

In this section, we check the model's ability to make predictions before any training has occurred. This is often referred to as a "sanity check" to ensure the model is functional.

## Model Prediction
We provide a sample input tensor `x` to the Multiple Linear Regression (MLR) model and evaluate its prediction. Please note that at this stage, the model has not undergone any training.


In [11]:
# Check the model gives data (NO TRAINING YET)
x = torch.tensor([8, 6, 10, 1.0, 16, 100])
y_pred = MLR_model(x)
print(y_pred)

ValueError: RNN: Expected input to be 2D or 3D, got 1D tensor instead

# Model Training Loop

In this section, we define the training loop for the Multiple Linear Regression (MLR) model. The training process involves multiple epochs, where the model learns from the training data to optimize its parameters and reduce the loss.

In [ ]:
t_losses = []
for epoch in range(EPOCH):
    for i, (inputs, targets) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = MLR_model(inputs)
        loss = criterion(outputs, targets)
        t_losses.append(loss.item())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        # scheduler.step()

        # Print training statistics
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{EPOCH}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


# Model Evaluation on Test Data

In this section, we evaluate the trained MLR model on the test dataset and calculate the test loss to assess the model's performance.

In [ ]:
# Evaluate the model on the test data
with torch.no_grad():
    total_loss = 0
    
    # Iterate over batches in the test loader
    for inputs, targets in test_loader:
        outputs = MLR_model(inputs)  # Forward pass to get model predictions
        loss = criterion(outputs, targets)  # Calculate the loss
        
        total_loss += loss.item()  # Accumulate the loss for all batches
    
    mean_loss = total_loss / len(test_loader)  # Calculate the mean loss over all batches
    print(f'Test Loss: {mean_loss:.4f}')

# Plotting Training Losses

In this section, we visualize the training progress by plotting the training losses. The loss values are plotted against the number of iterations, providing insight into the model's convergence during training.

In [ ]:
import matplotlib.pyplot as plt

# Plot the training losses
plt.plot(t_losses)
plt.xlabel("No. of Iterations")
plt.ylabel("Total Loss")
plt.show()

# Testing the Model

In this section, we evaluate the trained model by testing it on a set of case histories. The case histories include parameters and known outcomes for different locations.

## Case Histories Parameters
We define a list of case histories parameters, where each sub-list represents a location's specific parameters:

```python
cases = [
    [2.4, 4.8, 15, 1, 19, 12],   # Fornebu, Oslo
    [5, 5, 0, 1, 19, 16],       # Feria, Oslo
    [11.3, 16, 0, 1, 19, 35]    # Chicago, USA
]


In [ ]:
# Testing the model

# Case histories parameters
# Input is on the form [B (m), H (m), q (kPa), Rint, Gamma (kN/cu.m), Su (kPa)]

cases = [
    [2.4, 4.8, 15, 1, 19, 12],  # Fornebu, Oslo
    [5, 5, 0, 1, 19, 16],      # Feria, Oslo
    [11.3, 16, 0, 1, 19, 35]   # Chicago, USA
]


# Case histories outcomes
cases_op = [
    [1.03, 1.11, 0.97, 1.05],  # Fornebu, Oslo
    [1.02, 1.26, 1.03, 1.1],  # Feria, Oslo
    [1, 1.11, 0.95, 1]        # Chicago, USA
]

for i, each_case in enumerate(cases):
    y_pred_MLR = MLR_model(torch.Tensor(each_case))
    val = float(np.average(y_pred_MLR.detach()))
    cases_op[i].append(val)
    print(f'Avg. FoS for case_{i} is {val}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

species = ('Fornebu, Oslo', 'Feria, Oslo', 'Chicago, USA')

means = {
    "Eide's (1956)": [],
    "Terzaghi (1943)": [],
    "Simplified UBLA": [],
    "Modified Terzaghi": [],
    "FELA-FNN": []
}

# Assuming you have 'cases_op' defined elsewhere

for i, items in enumerate(means.items()):
    means[items[0]] += ([x[i] for x in cases_op])

x = np.arange(len(species))  # the label locations
width = 0.1  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(6.4, 5.5))

for attribute, measurement in means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    multiplier += 1

# Add some text for labels, title, and custom x-axis tick labels, etc.
ax.set_ylabel('FoS')

# Set y-axis limits
ax.set_ylim(0.5,1.5)

# Adjust the x-axis tick positions and labels
new_x = x + width * (len(means) - 1) / 2
ax.set_xticks(new_x)
ax.set_xticklabels(species, rotation=0, ha='center', fontsize=8, fontweight='bold')  # Adjust 'ha' to 'center'

# Adjust the legend location to 'upper right' and specify the number of columns (ncols)
legend = ax.legend(loc='upper right', ncols=1)

# Set the transparency of the legend background to 50%
legend.get_frame().set_alpha(0.5)

ax.axhline(y=1, color='red', linestyle='--', label='Y = 1')

# Set the chart title with a customizable font size
chart_title = "Comparison between the FoS predictions of different classical approaches and the FELA-FNN"
title_fontsize = 8  # You can adjust the font size as needed
ax.set_title(chart_title, fontsize=title_fontsize, fontweight='bold')  # Make the title bold

# Increase figure quality by adjusting the DPI
plt.savefig('your_figure_600_2.png', dpi=600)  # You can adjust the filename and DPI as needed

plt.show()

## Plotting Su vs. FoS

In [ ]:
''' 
    B:      Excavation width (m)
    H:      Excavation height (m)
    q:      Uniform surcharge at GS (kPa)
    Rint:   Interface strength reduction factor
    Gamma:  Soil unit weight (kN/cu.m)
    Su:     Undrained shear strength (kPa)
    
case_input = [B (m), H (m), q (kPa), Rint, Gamma (kN/cu.m), Su (kPa)]
y_pred_MLR = MLR_model(torch.Tensor(each_case))
LB_FoS = float(np.min(y_pred_MLR.detach()))
UB_FoS = float(np.max(y_pred_MLR.detach()))

'''
no_of_curve_points = 50
B = 5
H = 10
q = 20
Rint = 1
Gamma = 19
su = range(0, no_of_curve_points, 1)  # Changed the step size to 1

cases = []
for i in su:
    cases.append([B, H, q, Rint, Gamma, i])

for each_case in cases:
    y_pred_MLR = MLR_model(torch.Tensor(each_case))    
    LB_FoS = float(np.min(y_pred_MLR.detach()))
    print('LB FoS: ', LB_FoS)
    UB_FoS = float(np.min(y_pred_MLR.detach()))
    print('UB FoS: ',UB_FoS)
    Avg_FoS = float(np.average(y_pred_MLR.detach()))
    print(Avg_FoS)
    

In [ ]:
#cases
print('Nahh')

In [ ]:
''' 
    B:      Excavation width (m)
    H:      Excavation height (m)
    q:      Uniform surcharge at GS (kPa)
    Rint:   Interface strength reduction factor
    Gamma:  Soil unit weight (kN/cu.m)
    Su:     Undrained shear strength (kPa)
    
case_input = [B (m), H (m), q (kPa), Rint, Gamma (kN/cu.m), Su (kPa)]
'''

no_of_curve_points = 50
B = 5
H = 10
q = 20
Rint = 1
Gamma = 19
su = range(0, no_of_curve_points, 1)  # Changed the step size to 1

cases = []
for i in su:
    cases.append([B, H, q, Rint, Gamma, i])

for each_case in cases:
    y_pred_MLR = MLR_model(torch.Tensor(each_case))    
    print(float(min(y_pred_MLR.detach())))
    Avg_FoS = float(np.average(y_pred_MLR.detach()))
    print('Avg FoS: ', Avg_FoS)  # Print the average factor of safety